In [1]:
%pip install geopandas
%pip install contextily

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Elena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Elena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box
from pathlib import Path

In [15]:
# Directorio donde está el notebook
BASE_DIR = Path.cwd()
# Carpeta padre (un nivel arriba)
PARENT_DIR = BASE_DIR.parent

# Definición de directorios de carga
directorio_0 = PARENT_DIR / "08. Municipios BD"
# Definición de directorios de carga
directorio_3= PARENT_DIR / "13. Mallado Potencia Instalada"

## 1. Captura del Mallado Geográfico

In [16]:

# REPRODUCIMOS EL MALLADO GEOGRÁFICO PARA APLICARLO EN POTENCIA

# 1. Load municipios data y transformarlos a GeoDataFrame
# Cargar datos
df = pd.read_csv(f"{directorio_0}/MUNICIPIOS.csv", sep=';', encoding='latin-1')



df = df[~df["PROVINCIA"].isin(["Illes Balears", "Las Palmas", "Santa Cruz de Tenerife"])]
df = df[["PROVINCIA", "NOMBRE_ACTUAL", "POBLACION_MUNI", "LONGITUD_ETRS89", "LATITUD_ETRS89"]]
df["LONGITUD_ETRS89"] = pd.to_numeric(df["LONGITUD_ETRS89"].str.replace(",", ".", regex=False), errors="coerce")
df["LATITUD_ETRS89"]  = pd.to_numeric(df["LATITUD_ETRS89"].str.replace(",", ".", regex=False), errors="coerce")

# Convert df to GeoDataFrame
gdf_mun = gpd.GeoDataFrame(df,
    geometry=gpd.points_from_xy(df["LONGITUD_ETRS89"], df["LATITUD_ETRS89"]),
    crs="EPSG:4326"
)

# 2. CREAR MALLADO SOBRE ESPAÑA PENINSULAR

# Cargar límites de España desde Natural Earth
url = "https://naciscdn.org/naturalearth/50m/cultural/ne_50m_admin_0_countries.zip"
world = gpd.read_file(url)
esp = world[world["ADMIN"] == "Spain"].to_crs(gdf_mun.crs) #Cambia el CRS (sistema de referencia) al sistema que usa nuestro dataframe


# --- 1. Filtrar solo Península ---
esp_pen = esp[~esp["NAME"].isin(["Canary Islands", "Islas Baleares"])]
esp_pen_4326 = esp_pen.to_crs(4326)

# --- 2. Crear mallado 10x10 sobre la Península ---
minx, miny, maxx, maxy = esp_pen_4326.total_bounds
nx = ny = 10
dx, dy = (maxx - minx) / nx, (maxy - miny) / ny

cells = [
    box(minx + i * dx, miny + j * dy, minx + (i + 1) * dx, miny + (j + 1) * dy)
    for i in range(nx) for j in range(ny)
]

grid = gpd.GeoDataFrame({"cell_id": range(len(cells))}, geometry=cells, crs=4326)
grid = grid[grid.intersects(esp_pen_4326.unary_union)].reset_index(drop=True)



C:\Users\Elena\AppData\Local\Temp\ipykernel_7184\4267321319.py:43: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  grid = grid[grid.intersects(esp_pen_4326.unary_union)].reset_index(drop=True)


## 2. Cargar los datos de INSTALACIONES con COORDENADAS y Transforamar a GeoDAtAFrame

In [17]:
# Cargar datos de coordenadas de instalaciones
df_inst_coord = pd.read_csv(f"{directorio_3}/PRETOR_REDUCIDO_Coord.csv", sep=',', encoding='latin-1')
df_inst_coord["LONGITUD_ETRS89"] = pd.to_numeric(df_inst_coord["LONGITUD_ETRS89"].str.replace(",", ".", regex=False), errors="coerce")
df_inst_coord["LATITUD_ETRS89"]  = pd.to_numeric(df_inst_coord["LATITUD_ETRS89"].str.replace(",", ".", regex=False), errors="coerce")


# 2. Convert df to GeoDataFrame
gdf_inst= gpd.GeoDataFrame(
    df_inst_coord,
    geometry=gpd.points_from_xy(df_inst_coord["LONGITUD_ETRS89"], df_inst_coord["LATITUD_ETRS89"]),
    crs="EPSG:4326"
)



## 3. Asignar a CADA CELDA del MALLADO la POBLACIÓN conservando id_celda y calcular centroides

In [ ]:
# --- 3. Asignar municipios a las celdas ---

# Filtrar municipios dentro de la Península de nuestro dataframe de municipios
gdf_mun = gdf_mun[gdf_mun.within(esp_pen_4326.unary_union)].copy()


#Creo un datafrae de tipo geodataframe que añade a cada municipio la cell_id a la que pertenece
gdf_mun_cells = gpd.sjoin(gdf_mun, grid, predicate="within", how="left")

# --- 4. Calcular centro "ponderado por municipios" de cada celda y población total ---
# Pero — ese “centro” no es el centro geométrico real de la celda

#  En este caso, el punto central no depende del polígono de la celda del mallado.
#  Es simplemente la media de las coordenadas de los municipios dentro de esa celda.


# Por cada id de Celda se genera un nuevo dataframe con la población total y la latitud y longitud media de los municipios que contiene esa celda
mun_agg = (
    gdf_mun_cells.groupby("cell_id", as_index=False)
    .agg(
        POBLACION_TOTAL=("POBLACION_MUNI", "sum"),
        LAT_CENTRO=("LATITUD_ETRS89", "mean"),
        LON_CENTRO=("LONGITUD_ETRS89", "mean")
    )
)

# --- 5. Exportar CSV con coordenadas centrales y población total ---

# Guardar resultados
mun_agg.to_csv(f"{directorio_3}/MALLADO_Población.csv", index=False)


C:\Users\Elena\AppData\Local\Temp\ipykernel_7184\517946637.py:4: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gdf_mun = gdf_mun[gdf_mun.within(esp_pen_4326.unary_union)].copy()


## 4. Asignar a cada celda del MALLADO la POTENCIA INSTALADA POR tipo y AÑO

In [ ]:
# --- 3. Asignar potencia instalada a las celdas ---
gdf_inst = gdf_inst[gdf_inst.within(esp_pen_4326.unary_union)].copy()
gdf_inst_cells = gpd.sjoin(gdf_inst, grid, predicate="within", how="left")

# Nombres de columnas
date_col  = "Fecha"
power_col = "Potencia"
group_col = "Grupo"

resultados = []

for year in range(2019, 2025 + 1):
    cutoff = pd.Timestamp(year=year, month=6, day=30)

    # Asegura tipo datetime y filtra "activas" a 30 de junio del año
    df = gdf_inst_cells.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df[df[date_col] <= cutoff]

    # Estándar mínimo por si hay mayúsculas/minúsculas (Solar/Eolica)
    df[group_col] = df[group_col].astype(str).str.strip().str.title()

    # Agregación por celda y grupo
    agg = (df
           .groupby(["cell_id", group_col], as_index=False)
           .agg(Potencia=(power_col, "sum"))
          )

    # Pivot a columnas Solar / Eolica
    wide = (agg
            .pivot(index="cell_id", columns=group_col, values="Potencia")
            .fillna(0)
            .reset_index()
           )

    # Asegura columnas aunque falten ese año
    for col in ["Solar", "Eolico"]:
        if col not in wide.columns:
            wide[col] = 0

    # Renombra y añade columna Año
    wide = (wide
            .rename(columns={"Solar": "Pot_Solar_kW", "Eolico": "Pot_Eolico_kW"})
            .assign(Year=year)
           )

    resultados.append(wide)

# Concatenar todos los años en un único DataFrame
df_mallado_inst_todos = (pd.concat(resultados, ignore_index=True)
              .sort_values(["Year", "cell_id"])
              .reset_index(drop=True))

# Guardar resultados
df_mallado_inst_todos.to_csv(f"{directorio_3}/MALLADO_Potencia.csv", index=False)


C:\Users\Elena\AppData\Local\Temp\ipykernel_7184\2259408510.py:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gdf_inst = gdf_inst[gdf_inst.within(esp_pen_4326.unary_union)].copy()
